<a href="https://colab.research.google.com/github/RobertoAthos/big-data-college-study/blob/main/trabalho_big_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Comandos para realização do trabalho da matéria de Big Data com uso da biblioteca PySpark.

## <font color=red>Observação importante:</font>

<font color=yellow>Trabalho realizado com uso da biblioteca pandas não será aceito!</font>

## Upload do arquivo `imdb-reviews-pt-br.csv` para dentro do Google Colab

In [ ]:
!wget https://raw.githubusercontent.com/N-CPUninter/Big_Data/main/data/imdb-reviews-pt-br.zip -O imdb-reviews-pt-br.zip
!unzip imdb-reviews-pt-br.zip
!rm imdb-reviews-pt-br.zip

## Instalação manual das dependências para uso do pyspark no Google Colab

In [ ]:
!pip install pyspark

## Importar, instanciar e criar a SparkSession

In [ ]:
from pyspark.sql import SparkSession

appName = "PySpark Trabalho de Big Data"
master = "local"

spark = SparkSession.builder.appName(appName).master(master).getOrCreate()

## Criar spark dataframe do CSV utilizando o método read.csv do spark

In [ ]:
imdb_df = spark.read.csv('imdb-reviews-pt-br.csv',
                         header=True,
                         quote="\"",
                         escape="\"",
                         encoding="UTF-8")

# Questão 1

## Criar funções de MAP:
- Criar função para mapear o "sentiment" como chave e o "id" como valor do tipo inteiro

In [ ]:
def map1(x):
  # Coloque aqui o seu código para retornar a tupla necessária.
  # Apague a linha abaixo para iniciar seu código.
  pass

## Cria funções de REDUCE:

- Criar função de reduce para somar os IDs por "sentiment".

In [ ]:
def reduceByKey1(x,y):
  # Coloque aqui o seu código para retornar o resultado necessário.
  # Apague a linha abaixo para iniciar seu código.
  pass

## Aplicação do map/reduce e visualização do resultado

In [ ]:
# Coloque aqui a sua linha de código para aplicar o map/reduce no seu
# dataframe spark e realize o collect() ao final para visualizar os dados.


# Questão 2:

## Criar funções de MAP:
- Criar função para mapear o "sentiment" como chave e uma tupla com a soma das palavras de cada texto como valor.

In [ ]:
def map2(x):
  # Coloque aqui o seu código para retornar a tupla necessária.
  # Apague a linha abaixo para iniciar seu código.
  pass

## Cria funções de REDUCE:

- Criar função de reduce para somar o numero de palavras de cada texto português e inglês por "sentiment".

In [ ]:
def reduceByKey2(x,y):
  # Coloque aqui o seu código para retornar o resultado necessário.
  # Apague a linha abaixo para iniciar seu código.
  pass

## Aplicação do map/reduce e visualização do resultado

1. Aplicar o map/reduce no seu dataframe spark e realizar o collect() ao final
2. Selecionar os dados referentes aos textos negativos para realizar a subtração.
3. Realizar a subtração das contagens de palavras dos textos negativos para obter o resultado final

In [ ]:
# Coloque aqui suas linhas de código final
